# Data cleaning and preparation

##  Import Libraries  

In this step, we import all the Python libraries required for the project:  
- **pandas, numpy** – for data manipulation and numerical operations  
- **matplotlib** – for creating visualizations  
- **datetime** – for working with date and time data

In [ ]:
# Suppress FutureWarnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Date handling
from datetime import datetime

print("Libraries imported successfully.")


Libraries imported successfully.


## Load the Dataset

In this step, we load the *Lagos Air Pollution & Health Data* from the provided Excel file into a Pandas DataFrame.\
Using Pandas’ `read_excel()` function allows us to easily access and manipulate the dataset.\
After loading, we preview the first few rows to understand the structure of the data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/lagos_air_pollution_health_data.csv (1).xlsx'

# Load Excel file
df = pd.read_excel(file_path)

# Quick check to confirm data loaded
print(df.head())

           C       date  pm2_5    pm10    no2    so2     o3 hospital_id  \
0  I K E J A 2021-01-01  65.64  135.39  45.21  22.29  34.84  HOSP_IKE_1   
1      Ikeja 2021-01-01  76.49  116.65  50.01  10.20  32.19  HOSP_IKE_4   
2  I K E J A 2021-01-01  55.50  101.72  39.10  21.91  49.99  HOSP_IKE_1   
3      Ikeja 2021-01-01  56.37  138.53  46.40  28.13  24.89  HOSP_IKE_8   
4  I K E J A 2021-01-01  93.77  123.73  59.55  11.82  29.73  HOSP_IKE_3   

   respiratory_cases  avg_age_of_patients  weather_temperature  \
0               16.0                 44.0                 25.1   
1               12.0                 33.1                 25.3   
2               20.0                 33.5                 27.7   
3               15.0                 49.3                 32.8   
4               20.0                 45.4                 27.9   

   weather_humidity  wind_speed  rainfall_mm  population_density  \
0              40.0         3.1          2.2             22937.7   
1              5

##  Inspect the Dataset

In this step, we perform a quick inspection of the dataset to understand:
- The first few records (to get a feel for the data)
- The column names and their data types
- The overall size of the dataset\
This helps us confirm that the file was loaded correctly and provides an overview before cleaning or analysis.

In [ ]:
# View the first 5 rows
df.head()

,C,date,pm2_5,pm10,no2,so2,o3,hospital_id,respiratory_cases,avg_age_of_patients,weather_temperature,weather_humidity,wind_speed,rainfall_mm,population_density,industrial_activity_index
0,I K E J A,2021-01-01,65.64,135.39,45.21,22.29,34.84,HOSP_IKE_1,16.0,44.0,25.1,40.0,3.1,2.2,22937.7,0.87
1,Ikeja,2021-01-01,76.49,116.65,50.01,10.20,32.19,HOSP_IKE_4,12.0,33.1,25.3,53.5,3.8,2.2,21787.8,0.32
2,I K E J A,2021-01-01,55.50,101.72,39.10,21.91,49.99,HOSP_IKE_1,20.0,33.5,27.7,72.3,4.7,1.0,14798.0,0.78
3,Ikeja,2021-01-01,56.37,138.53,46.40,28.13,24.89,HOSP_IKE_8,15.0,49.3,32.8,62.9,4.8,8.2,18013.6,0.24
4,I K E J A,2021-01-01,93.77,123.73,59.55,11.82,29.73,HOSP_IKE_3,20.0,45.4,27.9,64.8,3.4,9.1,25980.3,0.30


In [ ]:
# Check basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258420 entries, 0 to 258419
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   C                          258392 non-null  object        
 1   date                       258397 non-null  datetime64[ns]
 2   pm2_5                      258417 non-null  float64       
 3   pm10                       258411 non-null  float64       
 4   no2                        258419 non-null  float64       
 5   so2                        258412 non-null  float64       
 6   o3                         258413 non-null  float64       
 7   hospital_id                258418 non-null  object        
 8   respiratory_cases          258413 non-null  float64       
 9   avg_age_of_patients        258404 non-null  float64       
 10  weather_temperature        258416 non-null  float64       
 11  weather_humidity           258414 non-null  float64 

In [ ]:
# Display the shape (rows, columns)
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

Dataset contains 258420 rows and 16 columns.


## Handle Missing Values

Before cleaning the dataset, it is important to identify where data is missing.\
In this step, we:
1. Display the number of missing values per column.
2. Fill numerical columns with the **median** (less affected by extreme values).
3. Fill categorical columns with the **mode** (most common value).
4. Forward-fill missing dates.

This ensures the dataset is complete and ready for analysis, while keeping the process transparent.

In [ ]:
# Display missing values per column
print("Missing values before cleaning:")
print(df.isnull().sum().sort_values(ascending=False))

Missing values before cleaning:
C                            28
date                         23
population_density           22
avg_age_of_patients          16
pm10                          9
so2                           8
o3                            7
respiratory_cases             7
weather_humidity              6
industrial_activity_index     5
wind_speed                    4
weather_temperature           4
pm2_5                         3
rainfall_mm                   3
hospital_id                   2
no2                           1
dtype: int64


In [ ]:
# Numerical columns – fill missing values with median
num_cols = df.select_dtypes(include='number').columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical columns – fill missing values with mode
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Fill missing dates using forward-fill method
df['date'] = df['date'].fillna(method='ffill')

# Extract year and month from the date column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Confirm that all missing values are handled
print("\nMissing values after cleaning:")
print(df.isnull().sum().sort_values(ascending=False))


Missing values after cleaning:
C                            0
date                         0
pm2_5                        0
pm10                         0
no2                          0
so2                          0
o3                           0
hospital_id                  0
respiratory_cases            0
avg_age_of_patients          0
weather_temperature          0
weather_humidity             0
wind_speed                   0
rainfall_mm                  0
population_density           0
industrial_activity_index    0
year                         0
month                        0
dtype: int64


/tmp/ipython-input-2245681498.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['date'] = df['date'].fillna(method='ffill')


## standardizing city column
Before analysis, we ensure our dataset is tidy and consistent:

Rename columns to meaningful names.

Standardize city names to avoid duplication due to inconsistent formatting (e.g., I K E J A → Ikeja).

Convert columns to appropriate data types for more efficient storage and faster processing.

In [ ]:
# Rename column 'c' to 'city'
df.rename(columns={'C': 'city'}, inplace=True)

# Standardize city names
city_map = {
    'I K E J A': 'Ikeja', 'ikeja': 'Ikeja', 'Ikeja': 'Ikeja',
    'YABA': 'Yaba', 'yaba': 'Yaba', 'Yaba': 'Yaba',
    'ajah': 'Ajah', 'A J A H': 'Ajah', 'Ajah': 'Ajah',
    'surulere': 'Surulere', 'SURULERE': 'Surulere', 'Surulere': 'Surulere',
    'LEKKI': 'Lekki', 'lekki': 'Lekki', 'Lekki': 'Lekki',
}
df['city'] = df['city'].map(city_map)

# Convert columns to appropriate data types
df['city'] = df['city'].astype('category')
df['hospital_id'] = df['hospital_id'].astype('category')
df['respiratory_cases'] = df['respiratory_cases'].astype(int)

# Check data types after changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258420 entries, 0 to 258419
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   city                       258420 non-null  category      
 1   date                       258420 non-null  datetime64[ns]
 2   pm2_5                      258420 non-null  float64       
 3   pm10                       258420 non-null  float64       
 4   no2                        258420 non-null  float64       
 5   so2                        258420 non-null  float64       
 6   o3                         258420 non-null  float64       
 7   hospital_id                258420 non-null  category      
 8   respiratory_cases          258420 non-null  int64         
 9   avg_age_of_patients        258420 non-null  float64       
 10  weather_temperature        258420 non-null  float64       
 11  weather_humidity           258420 non-null  float64 

In [ ]:
df.to_csv('/content/drive/MyDrive/Main_cleaned_air_pollution_data.csv', index=False)
print("Cleaned data saved successfully!")



Cleaned data saved successfully!
